# 03_M15plus_ACE_plus_CBD.ipynb
Replicates ACE-only vs ACE+CBD results for M15+: route-wide, rush windows, worst trips, crawl-share, top corridor improvements, and the all-day distribution shift.


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from IPython.display import display


In [ ]:
# ---------------------------
# Load & setup
# ---------------------------
DATA_PATH = "hunter-speeds-ace-labeled.csv"  # <- change if needed
df = pd.read_csv(DATA_PATH)

df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
df["Is Weekday"] = df["Timestamp"].dt.dayofweek.isin([0,1,2,3,4])
df["Corridor"] = df["Timepoint Stop Name"] + " → " + df["Next Timepoint Stop Name"]

In [ ]:
# Phase windows
ACE_ANNOUNCE   = pd.Timestamp("2024-06-17")  # announcement
ACE_FINE_START = pd.Timestamp("2024-08-16")  # ~60-day warning ends
CBD_START      = pd.Timestamp("2025-01-05")  # congestion pricing start

def phase_label(ts):
    if ts < ACE_ANNOUNCE:
        return "Pre-ACE"
    elif ACE_ANNOUNCE <= ts < ACE_FINE_START:
        return "ACE Warning (skip)"
    elif ACE_FINE_START <= ts < CBD_START:
        return "ACE only"
    else:
        return "ACE + CBD"

df["Phase"] = df["Timestamp"].apply(phase_label)
df["Phase"].value_counts(dropna=False)

In [ ]:
# Verify speed formula (sanity check)
if not df[["Road Distance","Average Travel Time","Average Road Speed"]].dropna().empty:
    v = df[["Road Distance","Average Travel Time","Average Road Speed"]].dropna().iloc[0]
    print("Speed formula check (mph ≈ 60*miles/min):",
          float(60*v["Road Distance"]/v["Average Travel Time"]), "vs", float(v["Average Road Speed"]))

In [ ]:
# Weighted helpers
def wavg(x, w):
    x, w = np.asarray(x), np.asarray(w)
    return np.average(x, weights=w) if len(x)>0 else np.nan

def wpercentile(x, w, q):
    x, w = np.asarray(x), np.asarray(w)
    if len(x)==0: return np.nan
    idx = np.argsort(x); x, w = x[idx], w[idx]
    cumw = np.cumsum(w); cutoff = q * w.sum()
    k = np.searchsorted(cumw, cutoff, side="left")
    k = int(min(max(k,0), len(x)-1))
    return float(x[k])

# Common filters
base = df[df["Is Weekday"] & (df["Phase"]!="ACE Warning (skip)")].copy()
allday = list(range(6,22))
am_hours = [7,8,9]
pm_hours = [16,17,18]

In [ ]:
def summarize_route(df_in, route_id, hours, label):
    sub = df_in[(df_in["Route ID"]==route_id) & df_in["Hour of Day"].isin(hours)]
    g = sub.groupby("Phase")
    out = {}
    for p, gg in g:
        out[p] = {
            "mean_mph": wavg(gg["Average Road Speed"], gg["Bus Trip Count"]),
            "p10_mph":  wpercentile(gg["Average Road Speed"], gg["Bus Trip Count"], 0.10),
            "crawl":    wavg((gg["Average Road Speed"]<5).astype(float), gg["Bus Trip Count"])
        }
    # Compact table for ACE-only vs ACE+CBD
    if "ACE only" in out and "ACE + CBD" in out:
        tbl = pd.DataFrame({
            "Window": [label],
            "Mean mph (ACE only)": [out["ACE only"]["mean_mph"]],
            "Mean mph (ACE + CBD)": [out["ACE + CBD"]["mean_mph"]],
            "P10 mph (ACE only)":   [out["ACE only"]["p10_mph"]],
            "P10 mph (ACE + CBD)":  [out["ACE + CBD"]["p10_mph"]],
            "Crawl<5 (ACE only)":   [out["ACE only"]["crawl"]],
            "Crawl<5 (ACE + CBD)":  [out["ACE + CBD"]["crawl"]],
        })
        display(tbl.round(3))
    return out

print("M15+ — Route-wide summary tables (ACE-only vs ACE+CBD):")
_ = summarize_route(base, "M15+", allday, "All-day (6–22)")
_ = summarize_route(base, "M15+", am_hours, "AM Rush (7–9)")
_ = summarize_route(base, "M15+", pm_hours, "PM Rush (16–18)")


In [ ]:
m15 = base[base["Route ID"]=="M15+"].copy()

def corridor_phase_means(data, hours):
    sub = data[data["Hour of Day"].isin(hours)]
    g = (sub.groupby(["Corridor","Direction","Phase"])
           .apply(lambda gg: wavg(gg["Average Road Speed"], gg["Bus Trip Count"]))
           .reset_index(name="Avg_mph"))
    piv = g.pivot_table(index=["Corridor","Direction"], columns="Phase", values="Avg_mph").reset_index()
    return piv

def top_corridors_by_change(data, hours, phase1="ACE only", phase2="ACE + CBD", top_n=5):
    piv = corridor_phase_means(data, hours)
    mask = piv[phase1].notna() & piv[phase2].notna()
    piv = piv[mask].copy()
    piv["%Δ (CBD vs ACE)"] = (piv[phase2] - piv[phase1]) / piv[phase1] * 100
    return piv.sort_values("%Δ (CBD vs ACE)", ascending=False).head(top_n)


In [ ]:
print("\nM15+ — Top corridor improvements (CBD vs ACE): AM Rush")
am_top = top_corridors_by_change(m15, am_hours, "ACE only", "ACE + CBD", top_n=5)
display(am_top.round(3))

print("\nM15+ — Top corridor improvements (CBD vs ACE): PM Rush")
pm_top = top_corridors_by_change(m15, pm_hours, "ACE only", "ACE + CBD", top_n=5)
display(pm_top.round(3))


In [ ]:
m15_allday = m15[m15["Hour of Day"].between(6,21)]

speeds_ace = m15_allday[m15_allday["Phase"]=="ACE only"]["Average Road Speed"].to_numpy()
speeds_cbd = m15_allday[m15_allday["Phase"]=="ACE + CBD"]["Average Road Speed"].to_numpy()

# Basic stats
mean_ace = np.average(speeds_ace) if len(speeds_ace) else np.nan
mean_cbd = np.average(speeds_cbd) if len(speeds_cbd) else np.nan
p10_ace = np.percentile(speeds_ace, 10) if len(speeds_ace) else np.nan
p10_cbd = np.percentile(speeds_cbd, 10) if len(speeds_cbd) else np.nan
crawl_ace = np.mean(speeds_ace<5) if len(speeds_ace) else np.nan
crawl_cbd = np.mean(speeds_cbd<5) if len(speeds_cbd) else np.nan

print("\nM15+ All-day distribution stats:")
print(" Mean mph — ACE only vs ACE+CBD:", round(mean_ace,3), "→", round(mean_cbd,3))
print(" P10 mph  — ACE only vs ACE+CBD:", round(p10_ace,3), "→", round(p10_cbd,3),
      " (Δ%:", round((p10_cbd - p10_ace)/p10_ace*100,2) if p10_ace>0 else np.nan, ")")
print(" Crawl<5 share — ACE only vs ACE+CBD:", round(crawl_ace,4), "→", round(crawl_cbd,4))


In [ ]:
plt.figure(figsize=(9,5))
plt.hist(speeds_ace, bins=40, alpha=0.5, label=f"ACE only (avg~{mean_ace:.2f}, P10~{p10_ace:.2f})", density=True)
plt.hist(speeds_cbd, bins=40, alpha=0.5, label=f"ACE+CBD (avg~{mean_cbd:.2f}, P10~{p10_cbd:.2f})", density=True)
plt.axvline(mean_ace, linestyle="--")
plt.axvline(mean_cbd, linestyle="--")
plt.axvline(p10_ace, linestyle=":")
plt.axvline(p10_cbd, linestyle=":")
plt.title("M15+ All-Day Speed Distribution — ACE only vs ACE+CBD")
plt.xlabel("Speed (mph)")
plt.ylabel("Density")
plt.legend()
plt.show()


In [ ]:
# Route-wide windows table
def route_window_table():
    rows = []
    for hrs, lbl in [(allday,"All-day"), (am_hours,"AM Rush"), (pm_hours,"PM Rush")]:
        sub = base[(base["Route ID"]=="M15+") & (base["Hour of Day"].isin(hrs))]
        d = {}
        for p, gg in sub.groupby("Phase"):
            d[p] = {
                "mean": wavg(gg["Average Road Speed"], gg["Bus Trip Count"]),
                "p10": wpercentile(gg["Average Road Speed"], gg["Bus Trip Count"], 0.10),
                "crawl": wavg((gg["Average Road Speed"]<5).astype(float), gg["Bus Trip Count"]),
            }
        if "ACE only" in d and "ACE + CBD" in d:
            rows.append({
                "Window": lbl,
                "Mean mph (ACE only)": d["ACE only"]["mean"],
                "Mean mph (ACE + CBD)": d["ACE + CBD"]["mean"],
                "P10 mph (ACE only)": d["ACE only"]["p10"],
                "P10 mph (ACE + CBD)": d["ACE + CBD"]["p10"],
                "Crawl<5 (ACE only)": d["ACE only"]["crawl"],
                "Crawl<5 (ACE + CBD)": d["ACE + CBD"]["crawl"],
            })
    return pd.DataFrame(rows)

route_win = route_window_table().round(3)
display(route_win)

# Save optional exports
route_win.to_csv("M15plus_RouteWindows_ACE_vs_CBD.csv", index=False)
am_top.to_csv("M15plus_TopCorridors_AM_CBD_vs_ACE.csv", index=False)
pm_top.to_csv("M15plus_TopCorridors_PM_CBD_vs_ACE.csv", index=False)

print("Exports written:")
print(" - M15plus_RouteWindows_ACE_vs_CBD.csv")
print(" - M15plus_TopCorridors_AM_CBD_vs_ACE.csv")
print(" - M15plus_TopCorridors_PM_CBD_vs_ACE.csv")


In [ ]:
# 03_M15plus_ACE_plus_CBD.ipynb
# Replicates ACE-only vs ACE+CBD results for M15+: route-wide, rush windows, worst trips, crawl-share,
# top corridor improvements, and the all-day distribution shift.

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# ---------------------------
# Load & setup
# ---------------------------
DATA_PATH = "hunter-speeds-ace-labeled.csv"  # <- change if needed
df = pd.read_csv(DATA_PATH)

df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
df["Is Weekday"] = df["Timestamp"].dt.dayofweek.isin([0,1,2,3,4])
df["Corridor"] = df["Timepoint Stop Name"] + " → " + df["Next Timepoint Stop Name"]

# Phase windows
ACE_ANNOUNCE   = pd.Timestamp("2024-06-17")  # announcement
ACE_FINE_START = pd.Timestamp("2024-08-16")  # ~60-day warning ends
CBD_START      = pd.Timestamp("2025-01-05")  # congestion pricing start

def phase_label(ts):
    if ts < ACE_ANNOUNCE:
        return "Pre-ACE"
    elif ACE_ANNOUNCE <= ts < ACE_FINE_START:
        return "ACE Warning (skip)"
    elif ACE_FINE_START <= ts < CBD_START:
        return "ACE only"
    else:
        return "ACE + CBD"

df["Phase"] = df["Timestamp"].apply(phase_label)

# Verify speed formula (sanity check):
if not df[["Road Distance","Average Travel Time","Average Road Speed"]].dropna().empty:
    v = df[["Road Distance","Average Travel Time","Average Road Speed"]].dropna().iloc[0]
    print("Speed formula check (mph ≈ 60*miles/min):",
          float(60*v["Road Distance"]/v["Average Travel Time"]), "vs", float(v["Average Road Speed"]))

# Weighted helpers
def wavg(x, w):
    x, w = np.asarray(x), np.asarray(w)
    return np.average(x, weights=w) if len(x)>0 else np.nan

def wpercentile(x, w, q):
    x, w = np.asarray(x), np.asarray(w)
    if len(x)==0: return np.nan
    idx = np.argsort(x); x, w = x[idx], w[idx]
    cumw = np.cumsum(w); cutoff = q * w.sum()
    k = np.searchsorted(cumw, cutoff, side="left")
    k = int(min(max(k,0), len(x)-1))
    return float(x[k])

# Common filters
base = df[df["Is Weekday"] & (df["Phase"]!="ACE Warning (skip)")].copy()
allday = list(range(6,22))
am_hours = [7,8,9]
pm_hours = [16,17,18]

# ---------------------------
# 1) Route-wide M15+ results (All-day, AM, PM): mean, P10, crawl share
# ---------------------------
def summarize_route(df_in, route_id, hours, label):
    sub = df_in[(df_in["Route ID"]==route_id) & df_in["Hour of Day"].isin(hours)]
    g = sub.groupby("Phase")
    out = {}
    for p, gg in g:
        out[p] = {
            "mean_mph": wavg(gg["Average Road Speed"], gg["Bus Trip Count"]),
            "p10_mph":  wpercentile(gg["Average Road Speed"], gg["Bus Trip Count"], 0.10),
            "crawl":    wavg((gg["Average Road Speed"]<5).astype(float), gg["Bus Trip Count"])
        }
    # display compact table for ACE-only vs ACE+CBD (primary comparison)
    if "ACE only" in out and "ACE + CBD" in out:
        tbl = pd.DataFrame({
            "Window": [label],
            "Mean mph (ACE only)": [out["ACE only"]["mean_mph"]],
            "Mean mph (ACE + CBD)": [out["ACE + CBD"]["mean_mph"]],
            "P10 mph (ACE only)": [out["ACE only"]["p10_mph"]],
            "P10 mph (ACE + CBD)": [out["ACE + CBD"]["p10_mph"]],
            "Crawl<5 (ACE only)": [out["ACE only"]["crawl"]],
            "Crawl<5 (ACE + CBD)": [out["ACE + CBD"]["crawl"]],
        })
        display(tbl.round(3))
    return out

print("M15+ — Route-wide summary tables (ACE-only vs ACE+CBD):")
_ = summarize_route(base, "M15+", allday, "All-day (6–22)")
_ = summarize_route(base, "M15+", am_hours, "AM Rush (7–9)")
_ = summarize_route(base, "M15+", pm_hours, "PM Rush (16–18)")

# ---------------------------
# 2) Top corridor improvements (CBD vs ACE), AM & PM
# ---------------------------
m15 = base[base["Route ID"]=="M15+"].copy()

def corridor_phase_means(data, hours):
    sub = data[data["Hour of Day"].isin(hours)]
    g = (sub.groupby(["Corridor","Direction","Phase"])
           .apply(lambda gg: wavg(gg["Average Road Speed"], gg["Bus Trip Count"]))
           .reset_index(name="Avg_mph"))
    piv = g.pivot_table(index=["Corridor","Direction"], columns="Phase", values="Avg_mph").reset_index()
    return piv

def top_corridors_by_change(data, hours, phase1="ACE only", phase2="ACE + CBD", top_n=5):
    piv = corridor_phase_means(data, hours)
    mask = piv[phase1].notna() & piv[phase2].notna()
    piv = piv[mask].copy()
    piv["%Δ (CBD vs ACE)"] = (piv[phase2] - piv[phase1]) / piv[phase1] * 100
    return piv.sort_values("%Δ (CBD vs ACE)", ascending=False).head(top_n)

print("\nM15+ — Top corridor improvements (CBD vs ACE): AM Rush")
am_top = top_corridors_by_change(m15, am_hours, "ACE only", "ACE + CBD", top_n=5)
display(am_top.round(3))

print("\nM15+ — Top corridor improvements (CBD vs ACE): PM Rush")
pm_top = top_corridors_by_change(m15, pm_hours, "ACE only", "ACE + CBD", top_n=5)
display(pm_top.round(3))

# ---------------------------
# 3) Worst-trip (P10) improvement & crawl-share elimination — All-day distribution plot
# ---------------------------
m15_allday = m15[m15["Hour of Day"].between(6,21)]

speeds_ace = m15_allday[m15_allday["Phase"]=="ACE only"]["Average Road Speed"].to_numpy()
speeds_cbd = m15_allday[m15_allday["Phase"]=="ACE + CBD"]["Average Road Speed"].to_numpy()

# Basic stats to print alongside the plot
mean_ace = np.average(speeds_ace) if len(speeds_ace) else np.nan
mean_cbd = np.average(speeds_cbd) if len(speeds_cbd) else np.nan
p10_ace = np.percentile(speeds_ace, 10) if len(speeds_ace) else np.nan
p10_cbd = np.percentile(speeds_cbd, 10) if len(speeds_cbd) else np.nan
crawl_ace = np.mean(speeds_ace<5) if len(speeds_ace) else np.nan
crawl_cbd = np.mean(speeds_cbd<5) if len(speeds_cbd) else np.nan

print("\nM15+ All-day distribution stats:")
print(" Mean mph — ACE only vs ACE+CBD:", round(mean_ace,3), "→", round(mean_cbd,3))
print(" P10 mph  — ACE only vs ACE+CBD:", round(p10_ace,3), "→", round(p10_cbd,3),
      " (Δ%:", round((p10_cbd - p10_ace)/p10_ace*100,2) if p10_ace>0 else np.nan, ")")
print(" Crawl<5 share — ACE only vs ACE+CBD:", round(crawl_ace,4), "→", round(crawl_cbd,4))

plt.figure(figsize=(9,5))
plt.hist(speeds_ace, bins=40, alpha=0.5, label=f"ACE only (avg~{mean_ace:.2f}, P10~{p10_ace:.2f})", density=True)
plt.hist(speeds_cbd, bins=40, alpha=0.5, label=f"ACE+CBD (avg~{mean_cbd:.2f}, P10~{p10_cbd:.2f})", density=True)
plt.axvline(mean_ace, linestyle="--")
plt.axvline(mean_cbd, linestyle="--")
plt.axvline(p10_ace, linestyle=":")
plt.axvline(p10_cbd, linestyle=":")
plt.title("M15+ All-Day Speed Distribution — ACE only vs ACE+CBD")
plt.xlabel("Speed (mph)"); plt.ylabel("Density"); plt.legend(); plt.show()

# ---------------------------
# 4) Optional: Export key tables for your deck
# ---------------------------
# Route-wide windows table
def route_window_table():
    rows = []
    for hrs, lbl in [(allday,"All-day"), (am_hours,"AM Rush"), (pm_hours,"PM Rush")]:
        sub = base[(base["Route ID"]=="M15+") & (base["Hour of Day"].isin(hrs))]
        d = {}
        for p, gg in sub.groupby("Phase"):
            d[p] = {
                "mean": wavg(gg["Average Road Speed"], gg["Bus Trip Count"]),
                "p10": wpercentile(gg["Average Road Speed"], gg["Bus Trip Count"], 0.10),
                "crawl": wavg((gg["Average Road Speed"]<5).astype(float), gg["Bus Trip Count"]),
            }
        if "ACE only" in d and "ACE + CBD" in d:
            rows.append({
                "Window": lbl,
                "Mean mph (ACE only)": d["ACE only"]["mean"],
                "Mean mph (ACE + CBD)": d["ACE + CBD"]["mean"],
                "P10 mph (ACE only)": d["ACE only"]["p10"],
                "P10 mph (ACE + CBD)": d["ACE + CBD"]["p10"],
                "Crawl<5 (ACE only)": d["ACE only"]["crawl"],
                "Crawl<5 (ACE + CBD)": d["ACE + CBD"]["crawl"],
            })
    return pd.DataFrame(rows)

route_win = route_window_table().round(3)
display(route_win)

# Save optional exports
route_win.to_csv("M15plus_RouteWindows_ACE_vs_CBD.csv", index=False)
am_top.to_csv("M15plus_TopCorridors_AM_CBD_vs_ACE.csv", index=False)
pm_top.to_csv("M15plus_TopCorridors_PM_CBD_vs_ACE.csv", index=False)

print("Exports written:")
print(" - M15plus_RouteWindows_ACE_vs_CBD.csv")
print(" - M15plus_TopCorridors_AM_CBD_vs_ACE.csv")
print(" - M15plus_TopCorridors_PM_CBD_vs_ACE.csv")
